<a href="https://colab.research.google.com/github/edwardhongwang/2025_Spring_Harvard_ES26_ES294_Humanity_and_AI/blob/main/Session_4_Train_GPT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 2025 Spring ES26/ES294 Create a simple Generative Pretraning Transformer (GPT)

# Part 0 Import necessary infrastructure and hyperparameter

In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# ----------------------------
# Hyperparameters
# ----------------------------

batch_size = 64   # how many independent sequences we process in parallel
block_size = 256  # maximum context length
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

print("Using device:", device)
print("batch_size:", batch_size, "block_size:", block_size)


Using device: cuda
batch_size: 64 block_size: 256


## **1) Imports & Hyperparameters**

**Key Points**:

1. **Imports**  
   - We use `import torch` for the PyTorch library, `import torch.nn as nn` for building neural network modules, and `from torch.nn import functional as F` for functions like `F.softmax`.

2. **Essential Hyperparameters**  
   - **batch_size = 64**: In each training step, we process 64 sequences in parallel.  
   - **block_size = 256**: The maximum “context length” the model can see for each position (it looks at up to 256 tokens behind the current token).  
   - **n_embd = 384**: The size (dimension) of each token’s vector embedding (how many numbers represent one token).  
   - **n_head = 6**: How many separate “heads” we use in multi-head attention. Each head focuses on different aspects of the context.  
   - **n_layer = 6**: We stack 6 Transformer blocks on top of each other. Each block has self-attention + feed-forward layers.  
   - **dropout = 0.2**: The fraction of neurons (20%) “dropped” to prevent overfitting.  
   - **learning_rate = 3e-4**: The step size when adjusting the model’s weights during training.  
   - **max_iters = 5000**: We plan to update the model 5000 times in total.  
   - **eval_interval = 500**: Every 500 updates, we will measure and print the training and validation losses.  
   - **eval_iters = 200**: How many batches we use to estimate the average loss each time we evaluate the model.  
   - **device**: 'cuda' if a GPU is available, otherwise 'cpu'.  

These hyperparameters shape how large the network is, how it trains, and how we evaluate performance along the way.

# Part 1 Exploring the dataset Tiny Shakespear


In [2]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

--2025-03-24 19:00:53--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.02s   

2025-03-24 19:00:53 (44.6 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
# read it in to inspect it
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  1115394


In [5]:
# let's look at the first 1000 characters
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [6]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [7]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [8]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [9]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [10]:
block_size = 8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [11]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([18]) the target: 47
when input is tensor([18, 47]) the target: 56
when input is tensor([18, 47, 56]) the target: 57
when input is tensor([18, 47, 56, 57]) the target: 58
when input is tensor([18, 47, 56, 57, 58]) the target: 1
when input is tensor([18, 47, 56, 57, 58,  1]) the target: 15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target: 47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target: 58


In [12]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets:
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
----
when input is [24] the target: 43
when input is [24, 43] the target: 58
when input is [24, 43, 58] the target: 5
when input is [24, 43, 58, 5] the target: 57
when input is [24, 43, 58, 5, 57] the target: 1
when input is [24, 43, 58, 5, 57, 1] the target: 46
when input is [24, 43, 58, 5, 57, 1, 46] the target: 43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target: 39
when input is [44] the target: 53
when input is [44, 53] the target: 56
when input is [44, 53, 56] the target: 1
when input is [44, 53, 56, 1] the target: 58
when input is [44, 53, 56, 1, 58] the target: 46
when input is [44, 53

# Part 2 Prepare the data for training


In [13]:
# For reproducibility
torch.manual_seed(1337)

# ----------------------------
# Load the text data
# ----------------------------
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# Identify the set of all unique characters in the text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print("Unique characters in vocab:", chars)
print("Vocab size:", vocab_size)

# Create mappings from characters to integers (and vice versa)
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }

encode = lambda s: [stoi[c] for c in s]  # encoder: string -> list of ints
decode = lambda l: ''.join([itos[i] for i in l])  # decoder: list of ints -> string

# ----------------------------
# Train / validation splits
# ----------------------------
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

print(f"Data has {len(data)} tokens total.")
print(f"Split into {len(train_data)} training tokens and {len(val_data)} validation tokens.")


Unique characters in vocab: ['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Vocab size: 65
Data has 1115394 tokens total.
Split into 1003854 training tokens and 111540 validation tokens.


## **2) Load & Prepare Data**

**Key Points**:

1. **Reading the Dataset**  
   - We open `input.txt` and read it into a single large string called `text`. This might be Shakespeare or any large corpus.

2. **Character Vocabulary**  
   - We extract the unique characters that appear in the text and sort them. For example, if the text has letters, punctuation, and spaces, those all count as characters.  
   - **vocab_size** is how many unique characters we end up with.

3. **Mapping Characters to Integers**  
   - **stoi**: (string-to-integer) a dictionary mapping each character to a unique integer ID (0, 1, 2, …).  
   - **itos**: (integer-to-string) the reverse mapping, so we can decode predicted integers back to text.

4. **Encoding & Decoding**  
   - An **encode** function that turns a text string into a list of integer IDs.  
   - A **decode** function that does the opposite (IDs → text).

5. **Train/Validation Split**  
   - We take 90% of the text for training (stored in `train_data`) and the final 10% for validation (in `val_data`).  
   - This ensures we can evaluate how well the model generalizes to unseen text.


# Part 3 Batch function and loss estimation

In [14]:
def get_batch(split):
    """
    Returns a batch of x and y data of shape (batch_size, block_size).
    x[i] is a context of `block_size` tokens.
    y[i] is those same tokens shifted by 1 (the "next token").
    """
    data_source = train_data if split == 'train' else val_data
    ix = torch.randint(len(data_source) - block_size, (batch_size,))
    x = torch.stack([data_source[i:i+block_size] for i in ix])
    y = torch.stack([data_source[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss(model):
    """
    Evaluate the model on both training and validation splits
    to see how the average loss is doing.
    """
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            _, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out



## **3) Batching & Loss Estimation**

**Key Points**:

1. **get_batch(split)**  
   - **batch_size = 64**: We pick 64 random starting positions in the text (from training or validation portion).  
   - **block_size = 256**: From each starting position, we slice 256 tokens as the input (`x`).  
   - We also take the next 256 tokens as the “target” (`y`), effectively shifting by 1. So if `x` = “HELLO”, `y` = “ELLO?” (the next character is what we want to predict).

2. **estimate_loss(model)**  
   - Temporarily switches the model to evaluation mode (`model.eval()`).  
   - **eval_iters = 200**: For each split ('train' and 'val'), we run 200 batches through the model and compute the average cross-entropy loss.  
   - Returns the mean loss for both training and validation.  
   - Switches the model back to training mode (`model.train()`) so dropout is re-enabled for training.


# Part 4a Single Attention Head


In [15]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

## 4a) **Single Attention Head**

1. **Self-Attention**  
   - For each token, we compute `query`, `key`, and `value` vectors (dimension `head_size`).  

2. **Masking for Causality**  
   - We use a lower-triangular matrix to ensure each token can only attend to positions before it.  
   - **block_size = 256** determines the shape of that mask.

3. **Dot-Product Attention**  
   - An attention score matrix `(B, T, T)` is formed by `query @ key^T`.  
   - We apply `softmax` so each token “weights” how much it attends to each other token in the context.

4. **Dimension**  
   - If **n_embd = 384** and **n_head = 6**, then each head might have `head_size = 384 // 6 = 64`.

5. **Dropout = 0.2**  
   - Applied to the attention weights to help the model generalize.


# Part 4b Multi-Head Attention

In [16]:
class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

## 4b) **Multi-Head Attention**

1. **Parallel Heads**  
   - We create **n_head = 6** separate “Head” modules. Each can learn to focus on different patterns in the sequence.

2. **Concatenation**  
   - We combine the outputs of all heads along the final dimension. If each head outputs `(B, T, 64)`, we end up with `(B, T, 384)` (since `64 * 6 = 384`).

3. **Projection**  
   - A final linear layer maps `(B, T, 384)` back to `(B, T, 384)`, preserving the original embedding dimension.

4. **Dropout**  
   - Again, we use a dropout of 0.2 for regularization.


# Part 4c Feed Forward network (Fully connected Neural Network)

In [17]:
class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)


## 4c) **Feed Forward**

1. **Two-Layer MLP**  
   - First layer expands from **n_embd = 384** to **4 * n_embd = 1536**, then we apply a ReLU.  
   - Second layer brings it back down to 384.

2. **Per-Token Operation**  
   - The feed forward network acts on each token’s representation independently (after the attention is done).

3. **Dropout = 0.2**  
   - Applied to the output to avoid overfitting.


# Part 4d Transformer Block

In [18]:
class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

## 4d) **Transformer Block**

1. **Self-Attention + Feed Forward**  
   - Each block first does multi-head attention, then a feed forward step.  
   - We keep residual connections: `x = x + attention(...)`, then `x = x + feedforward(...)`.

2. **LayerNorm**  
   - **LayerNorm** helps stabilize training. We apply it before each sub-layer in this “pre-norm” design.

3. **n_layer = 6**  
   - We stack 6 of these blocks in the main model.

# Part 5 GPT Model

In [19]:
class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()

        # Token and Position embeddings
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)

        # Stacked Transformer blocks
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)           # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)  # final linear to vocab

        # Weight init
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        """
        idx: (B, T) integer tensor
        targets: (B, T) integer tensor for language modeling
        """
        B, T = idx.shape

        # Index into embeddings (token + position)
        tok_emb = self.token_embedding_table(idx)                # (B, T, n_embd)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T, n_embd)
        x = tok_emb + pos_emb                                    # (B, T, n_embd)

        # Pass through all the Transformer blocks
        x = self.blocks(x)  # (B, T, n_embd)
        x = self.ln_f(x)    # (B, T, n_embd)

        # Final logits
        logits = self.lm_head(x)  # (B, T, vocab_size)

        loss = None
        if targets is not None:
            # Flatten the logits and targets to 2D and 1D for cross_entropy
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        """
        Takes a context idx of shape (B, T) and generates up to max_new_tokens.
        """
        for _ in range(max_new_tokens):
            # Crop the context to the last block_size tokens
            idx_cond = idx[:, -block_size:]

            # Get the predictions
            logits, _ = self(idx_cond)

            # Focus only on the last time step
            logits = logits[:, -1, :]   # (B, vocab_size)

            # Softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, vocab_size)

            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)

            # Append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T+1)

        return idx


## **5) GPTLanguageModel**

**Key Points**:

1. **Token + Position Embeddings**  
   - We have an **Embedding** layer for tokens (size **vocab_size x n_embd**).  
   - Another **Embedding** for positions (size **block_size x n_embd**). We add these together to give each token a sense of “where” it is in the sequence.

2. **Stack of Blocks**  
   - We build **n_layer = 6** Transformer blocks in a row. Each block is the “attention + feed-forward + residual” combo.

3. **Final Layer Norm & Linear Head**  
   - A **LayerNorm** normalizes the final output.  
   - The linear “LM head” maps `(B, T, 384)` to `(B, T, vocab_size)` so we have a probability distribution over each possible next character.

4. **Loss Computation**  
   - If we are given a `targets` tensor of shape `(B, T)`, we flatten both the logits and targets to compare them with `F.cross_entropy`.  
   - This is how we train the model to predict the correct next character.

5. **generate(...)**  
   - A method for text generation: we repeatedly feed the current sequence to the model, pick the next token from the distribution, and append it.  
   - We keep going until we’ve generated as many tokens as we want.


# Part 6 Traning Loop


In [20]:
# Instantiate the model
model = GPTLanguageModel()
model = model.to(device)
print(f"\nNumber of parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")

# Create an optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

# ----------------------------
# Training loop
# ----------------------------
for iter in range(max_iters):

    # Evaluate the loss on train/val periodically
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(model)
        print(f"Step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # Get a batch
    xb, yb = get_batch('train')

    # Forward pass & compute loss
    logits, loss = model(xb, yb)

    # Backward pass
    optimizer.zero_grad(set_to_none=True)
    loss.backward()

    # Gradient descent step
    optimizer.step()

print("Training complete!")



Number of parameters: 10.69M
Step 0: train loss 4.1993, val loss 4.1878
Step 500: train loss 2.5320, val loss 2.5648
Step 1000: train loss 2.4552, val loss 2.4972
Step 1500: train loss 2.4580, val loss 2.4544
Step 2000: train loss 2.4251, val loss 2.4151
Step 2500: train loss 2.4428, val loss 2.4239
Step 3000: train loss 2.4038, val loss 2.3765
Step 3500: train loss 2.3900, val loss 2.4096
Step 4000: train loss 2.3742, val loss 2.3592
Step 4500: train loss 2.3469, val loss 2.3518
Step 4999: train loss 2.2858, val loss 2.3399
Training complete!



## **6) Training Loop**

**Key Points**:

1. **max_iters = 5000**  
   - We run 5000 steps of gradient updates.

2. **eval_interval = 500**  
   - Every 500 steps, we call `estimate_loss(model)` to print out the average training and validation losses. This helps us see progress and check if we’re overfitting.

3. **get_batch('train')**  
   - We fetch a batch of size **batch_size = 64** and length **block_size = 256** from the training data.

4. **Forward & Backward**  
   - `model(xb, yb)` returns the loss. We call `loss.backward()` to compute gradients, and `optimizer.step()` to update the weights.

5. **AdamW with learning_rate=3e-4**  
   - A commonly used optimizer for Transformers, which adjusts learning rate and includes a weight decay term.

6. **Loss Logging**  
   - We print statements like “Step 0: train loss 2.85, val loss 2.90” so we know how training is going.


# Part 7 Use the model for text generation

In [21]:
# Example prompt (feel free to change the text below)
prompt = "Fawwaaz Habbal King of"

In [22]:


# 1) Encode the prompt string into token IDs
context_ids = encode(prompt)

# 2) Convert to a Torch tensor (and move to GPU if available).
# Shape is (1, length_of_prompt) so we have batch_size=1
context = torch.tensor(context_ids, dtype=torch.long, device=device).unsqueeze(0)

# 3) Generate text from that context
generated_ids = model.generate(context, max_new_tokens=200)[0].tolist()

# 4) Decode the entire resulting token list back into text
generated_text = decode(generated_ids)
print(generated_text)

Fawwaaz Habbal King of, les ith.

Whe firen thishes ry cuygo?

Mome ashoy boele chay harle tre, thae to usspkesdings:
Heow and uti ouns ingoarse moye chan des! is, by
At whoue hidey gre;
Whis IUKhiss and furfia!
Oe cry hal



## **7) Text Generation**

**Key Points**:

1. **Context**  
   - We can start with a custom prompt (e.g., `"Fawwaaz Habbal King of"`).  
   - The model needs an initial sequence to predict from.

2. **Repetitive Sampling**  
   - Each generation step:  
     - We feed in the last **block_size = 256** tokens (or fewer if the sequence is shorter).  
     - The model outputs logits for the next token.  
     - We use `F.softmax` to convert logits to probabilities, then pick one token using `torch.multinomial`.

3. **Sampling vs. Argmax**  
   - **Sampling** from probabilities adds creativity. If we always took the largest logit, the text might become repetitive or stuck.

4. **decode(...)**  
   - Converts a list of generated token IDs back to a string so we can read the generated text.

5. **max_new_tokens**  
   - We specify how many tokens to generate, e.g., 300 or 500. The model stops once it has produced that many additional tokens.


#This Notebook draws code snipets from Dr Andrej Karpathy' orignal video and repo:
 https://www.youtube.com/watch?v=kCc8FmEb1nY&t=1362s,
 https://github.com/karpathy/ng-video-lecture/blob/master/gpt.py

## modified and annotated by Edward Wang for class demo